In [1]:
!pip install datasets --quiet
!pip install torch torchvision --quiet
!pip install pytorch_transformers --quiet

# !pip install nlp --quiet
!pip install transformers --quiet

     |████████████████████████████████| 184kB 19.3MB/s 
     |████████████████████████████████| 20.7MB 1.2MB/s 
     |████████████████████████████████| 112kB 53.9MB/s 
     |████████████████████████████████| 245kB 61.4MB/s 
     |████████████████████████████████| 184kB 18.0MB/s 
     |████████████████████████████████| 1.2MB 39.9MB/s 
     |████████████████████████████████| 133kB 58.8MB/s 
     |████████████████████████████████| 890kB 57.9MB/s 
     |████████████████████████████████| 71kB 11.5MB/s 
     |████████████████████████████████| 7.5MB 58.2MB/s 
ERROR: botocore 1.20.14 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
     |████████████████████████████████| 1.8MB 21.4MB/s 
     |████████████████████████████████| 3.2MB 52.0MB/s 


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset

In [3]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

In [4]:
dataset = load_dataset('amazon_reviews_multi', 'en')
print(dataset.keys())

Dataset amazon_reviews_multi downloaded and prepared to /root/.cache/huggingface/datasets/amazon_reviews_multi/en/1.0.0/f3357bd271e187385a38574fe31b8fb10055303f67fa9fce55e84d08c4870efd. Subsequent calls will reuse this data.
dict_keys(['train', 'validation', 'test'])


In [5]:
dataset['test'][0] # create dataloader which tokenizes text, and sends stars. maybe filter them?

{'language': 'en',
 'product_category': 'apparel',
 'product_id': 'product_en_0902516',
 'review_body': 'These are AWFUL. They are see through, the fabric feels like tablecloth, and they fit like children’s clothing. Customer service did seem to be nice though, but I regret missing my return date for these. I wouldn’t even donate them because the quality is so poor.',
 'review_id': 'en_0199937',
 'review_title': 'Don’t waste your time!',
 'reviewer_id': 'reviewer_en_0097389',
 'stars': 1}

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

In [7]:
device = torch.device('cuda')

In [8]:
MAX_SEQ_LENGTH = 128

class AmazonDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        original_data = self.dataset[idx]
        text = str(original_data['review_title']) + str(original_data['review_body'])
        label = int(original_data['stars']) - 1

        input_ids = self.tokenizer(text, padding='max_length', truncation=True, max_length=MAX_SEQ_LENGTH, return_tensors='pt').input_ids

        return input_ids[0], torch.tensor(MAX_SEQ_LENGTH, dtype=torch.long), torch.tensor(label, dtype=torch.long)


In [9]:
train_ds = AmazonDataset(dataset['train'], tokenizer)
test_ds = AmazonDataset(dataset['test'], tokenizer)
val_ds = AmazonDataset(dataset['validation'], tokenizer)
print(len(train_ds), len(test_ds), len(val_ds))

200000 5000 5000


In [18]:
BATCH_SIZE = 64
train_data = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_data = torch.utils.data.DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)
val_data = torch.utils.data.DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

In [19]:
from tqdm import tqdm

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model.classifier = nn.Linear(768, 5)
model = model.to(torch.device('cuda'))

lr = 2e-5
criterion = nn.CrossEntropyLoss() # maybe rebalance it later
optimizer = optim.Adam(model.parameters(), lr=lr)

def train(dataloader):
  model.train()
  total_loss = 0.0
  total_acc = 0.0
  total_examples = 0
  for i, (X, _, lbl) in enumerate(tqdm(dataloader)):
    optimizer.zero_grad()
    X = X.to(device)
    lbl = lbl.to(device)
    y_out = model(X).logits
    loss = criterion(y_out, lbl)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
    total_acc += torch.sum(torch.argmax(y_out, dim=1) == lbl).item()
    total_examples += len(X)
    if i % 100 == 0:
      print(i, total_loss / (i+1), total_acc * 100 / total_examples)
  print(total_loss / len(dataloader), total_acc * 100 / total_examples)

def test(dataloader):
  model.eval()
  total_loss = 0.0
  total_acc = 0.0
  total_examples = 0
  for X, _, lbl in tqdm(dataloader):
    X = X.to(device)
    lbl = lbl.to(device)
    y_out = model(X).logits
    loss = criterion(y_out, lbl)
    total_loss += loss.item()
    total_acc += torch.sum(torch.argmax(y_out, dim=1) == lbl).item()
    total_examples += len(X)
  print(total_loss / len(dataloader), total_acc * 100 / total_examples)

In [20]:
tqdm._instances.clear()
for e in range(3):
  print("_________TRAIN %d_________"%e)
  train(train_data)
  print("_________TEST_________")
  test(test_data)
test(val_data)

  0%|          | 0/3125 [00:00<?, ?it/s]

_________TRAIN 0_________


  0%|          | 1/3125 [00:00<41:08,  1.27it/s]

0 1.644667625427246 18.75


  3%|▎         | 101/3125 [01:19<38:22,  1.31it/s]

...

0.7020596162414551 69.552
_________TEST_________


  0%|          | 0/79 [00:00<?, ?it/s]

0.8092437854296044 65.46


100%|██████████| 79/79 [00:21<00:00,  3.73it/s]

0.826512254491637 64.6


In [21]:
# 64.6 isn't bad. let's see some examples or put our own

In [28]:
text = "This product is really good. The camera is fine. The speed is good."
xx = tokenizer(text, padding='max_length', truncation=True, max_length=MAX_SEQ_LENGTH, return_tensors='pt').input_ids.to(device)
yy = model(xx)
print(torch.argmax(yy.logits).item() + 1, "star(s)")

5 star(s)


In [29]:
text = "This product is good. The body is okay."
xx = tokenizer(text, padding='max_length', truncation=True, max_length=MAX_SEQ_LENGTH, return_tensors='pt').input_ids.to(device)
yy = model(xx)
print(torch.argmax(yy.logits).item() + 1, "star(s)")

4 star(s)


In [30]:
text = "Okay product. I expected more. Nothing special but not bad either"
xx = tokenizer(text, padding='max_length', truncation=True, max_length=MAX_SEQ_LENGTH, return_tensors='pt').input_ids.to(device)
yy = model(xx)
print(torch.argmax(yy.logits).item() + 1, "star(s)")

3 star(s)


In [31]:
text = "Bad product. Not the worst product but pretty bad."
xx = tokenizer(text, padding='max_length', truncation=True, max_length=MAX_SEQ_LENGTH, return_tensors='pt').input_ids.to(device)
yy = model(xx)
print(torch.argmax(yy.logits).item() + 1, "star(s)")

2 star(s)


In [32]:
text = "The camera is bad. The body shape is all wrong. They couldn't get anything right."
xx = tokenizer(text, padding='max_length', truncation=True, max_length=MAX_SEQ_LENGTH, return_tensors='pt').input_ids.to(device)
yy = model(xx)
print(torch.argmax(yy.logits).item() + 1, "star(s)")

1 star(s)
